In [3]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\HP\Desktop\NNDL_Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.optim as optim

In [11]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [3]:
# Define transformations (ResNet requires specific normalization)
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [5]:
# Load dataset
full_train_data = datasets.ImageFolder(root="C://Users//HP//Desktop//NNDL_Project//train", transform=transform)
test_data = datasets.ImageFolder(root="C://Users//HP//Desktop//NNDL_Project//test", transform=transform)

In [6]:
# Train-validation split (80% train, 20% validation)
train_size = int(0.8 * len(full_train_data))
val_size = len(full_train_data) - train_size
train_data, val_data = random_split(full_train_data, [train_size, val_size])


In [7]:
# Create DataLoaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

In [8]:
# Load Pretrained ResNet18 Model
model = models.resnet18(pretrained=True)


c:\Users\HP\Desktop\NNDL_Project\venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\HP\Desktop\NNDL_Project\venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [9]:
# Modify the last layer for 7 emotion classes
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(full_train_data.classes))  # 7 output classes

In [12]:
# Move model to GPU if available
model = model.to(device)

In [13]:
# Define Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [21]:
from sklearn.metrics import precision_score, recall_score, accuracy_score
from tqdm import tqdm  # Import tqdm for progress bars

import torch

# Define the model again (same as before)
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(full_train_data.classes))  # 7 output classes

# Load the model's state dict from epoch 5
checkpoint_path = "resnet18_emotion_epoch_5.pth"
model.load_state_dict(torch.load(checkpoint_path))

# Move the model to the correct device
model = model.to(device)

# Define the optimizer and loss function again (since we need them for training)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# Start training from epoch 6
start_epoch = 5
num_epochs = 20  # Continue training for 10 epochs in total (or adjust as needed)

# Training Loop from Epoch 6
for epoch in range(start_epoch, num_epochs):
    model.train()
    train_loss = 0
    
    for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}", total=len(train_loader)):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    # Validation Loop
    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Validating Epoch {epoch+1}", total=len(val_loader)):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Calculate Accuracy, Precision, and Recall
    val_accuracy = 100 * correct / total
    val_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
    val_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)

    print(f"Epoch {epoch+1}, Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}, "
          f"Val Accuracy: {val_accuracy:.2f}%, Val Precision: {val_precision:.2f}, Val Recall: {val_recall:.2f}")

    # Save model after each epoch
    torch.save(model.state_dict(), f"resnet18_emotion_epoch_{epoch+1}.pth")

# Final Evaluation on Test Set
model.eval()
correct = 0
total = 0
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Evaluating Test Set", total=len(test_loader)):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

test_accuracy = 100 * correct / total
test_precision = precision_score(all_labels, all_preds, average='weighted', zero_division=1)
test_recall = recall_score(all_labels, all_preds, average='weighted', zero_division=1)

print(f"Test Accuracy: {test_accuracy:.2f}%, Test Precision: {test_precision:.2f}, Test Recall: {test_recall:.2f}")


Validating Epoch 6: 100%|██████████| 180/180 [00:19<00:00,  9.21it/s]


Epoch 6, Train Loss: 0.4361, Val Loss: 1.2710, Val Accuracy: 63.41%, Val Precision: 0.63, Val Recall: 0.63


Validating Epoch 7: 100%|██████████| 180/180 [00:18<00:00,  9.68it/s]


Epoch 7, Train Loss: 0.2768, Val Loss: 1.5561, Val Accuracy: 61.39%, Val Precision: 0.62, Val Recall: 0.61


Validating Epoch 8: 100%|██████████| 180/180 [00:23<00:00,  7.56it/s]


...

Epoch 19, Train Loss: 0.0677, Val Loss: 2.2556, Val Accuracy: 62.24%, Val Precision: 0.63, Val Recall: 0.62


Validating Epoch 20: 100%|██████████| 180/180 [00:24<00:00,  7.30it/s]


Epoch 20, Train Loss: 0.0719, Val Loss: 2.4082, Val Accuracy: 61.67%, Val Precision: 0.61, Val Recall: 0.62


Evaluating Test Set: 100%|██████████| 225/225 [00:29<00:00,  7.55it/s]

Test Accuracy: 61.44%, Test Precision: 0.60, Test Recall: 0.61


In [18]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


You should consider upgrading via the 'C:\Users\HP\Desktop\NNDL_Project\venv\Scripts\python.exe -m pip install --upgrade pip' command.
